In [1]:
from futu import *
import time
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
############################ Global Configuration Variables ############################
FUTU_OPEND_ADDRESS = '127.0.0.1'  # OpenD listening address
FUTU_OPEND_PORT = 11111  # OpenD listening port
TRADING_ENVIRONMENT = TrdEnv.SIMULATE  # Trading environment: REAL / SIMULATE
TRADING_MARKET = TrdMarket.FUTURES  # Transaction market authority, used to filter accounts

In [3]:
# Parameters
symbol = "HK.HTImain"
QUANTITY = 1  # Number of contracts
LOOKBACK_PERIOD = 5  # Number of candles to calculate the moving average

# Initialize contexts
quote_context = OpenQuoteContext(host=FUTU_OPEND_ADDRESS, port=FUTU_OPEND_PORT)
# trade_context = OpenSecTradeContext(
#     filter_trdmarket=TRADING_MARKET,
#     host=FUTU_OPEND_ADDRESS,
#     port=FUTU_OPEND_PORT,
#     security_firm=SecurityFirm.FUTUSECURITIES
# )
future_context = OpenFutureTradeContext(host=FUTU_OPEND_ADDRESS, port=FUTU_OPEND_PORT, is_encrypt=None, security_firm=SecurityFirm.FUTUSECURITIES)

2025-01-06 20:02:00,164 | 59605 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=16524142
2025-01-06 20:02:00,170 | 59605 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=16524142


In [43]:
# Track execution details
executed_today = False
last_trade_date = None
entry_price = None
position_side = None  # TrdSide.BUY or TrdSide.SELL

STOP_LOSS_PERCENT = 0.01  # 1% stop loss
TAKE_PROFIT_PERCENT = 0.02  # 2% take profit

# Subscribe to daily candlestick data
def subscribe_to_klines(symbol):
    ret, data = quote_context.subscribe(symbol, [SubType.K_DAY])
    if ret != RET_OK:
        print(f"Failed to subscribe to daily candlestick data: {data}")
        return False
    print(f"Subscribed to daily candlestick data for {symbol}")
    return True

def fetch_historical_data(symbol, period=LOOKBACK_PERIOD):
    """Fetch historical daily K-line data for the symbol."""
    ret, data = quote_context.get_cur_kline(symbol, period + 1, KLType.K_DAY)
    if ret != RET_OK:
        print(f"Failed to fetch historical data: {data}")
        return None
    return data

def calculate_mean(data):
    """Calculate the moving average from historical data."""
    return data['close'].iloc[:-1].mean()  # Exclude the last row (current price)

def trading_logic():
    """Execute the trading strategy."""
    global executed_today, last_trade_date

    # Get the current date
    current_date = datetime.now().date()

    # Reset execution tracker if the day has changed
    if current_date != last_trade_date:
        executed_today = False
        last_trade_date = current_date
        entry_price = None
        position_side = None

    if executed_today:
        print("Trade already executed today. Skipping.")
        monitor_position()
        return

    # Fetch historical data
    data = fetch_historical_data(SYMBOL, LOOKBACK_PERIOD)
    if data is None:
        return

    current_price = data['close'].iloc[-1]  # Latest close price
    moving_average = calculate_mean(data)

    print(f"Current price: {current_price}, {LOOKBACK_PERIOD} days Moving average: {moving_average}")

    # Determine trade signal
    if current_price > moving_average:
        print("Sell signal triggered")
        #executed_today = place_trade(TrdSide.SELL, current_price)  # Use TrdSide.SELL
    elif current_price < moving_average:
        print("Buy signal triggered")
        #executed_today = place_trade(TrdSide.BUY, current_price)  # Use TrdSide.BUY


def place_trade(trd_side, execution_price):
    """Place a market order, track its execution, and display execution details."""
    global entry_price, position_side

    # Place the trade
    ret, data = trade_context.place_order(
        price=None,  # Market order
        qty=QUANTITY,
        code=SYMBOL,
        trd_side=trd_side,
        order_type=OrderType.MARKET,
        trd_env=TRADING_ENVIRONMENT
    )
    if ret != RET_OK:
        print(f"Failed to place {trd_side.name} order: {data}")
        return False

    # Print confirmation of order placement
    order_id = data['order_id']
    print(f"{trd_side.name} order placed successfully with Order ID: {order_id}")

    # Track execution
    while True:
        ret, order_status = trade_context.get_order_list(trd_env=TRADING_ENVIRONMENT)
        if ret != RET_OK:
            print(f"Failed to fetch order status: {order_status}")
            break

        # Find the order by ID and check its status
        order = order_status[order_status['order_id'] == order_id]
        if not order.empty:
            status = order.iloc[0]['status']
            if status in ['FILLED', 'EXECUTED']:
                print(f"{trd_side.name} order executed successfully for Order ID: {order_id}")

                # Track entry details
                entry_price = execution_price  # Update entry price
                position_side = trd_side  # Update position side
                stop_loss_price, take_profit_price = calculate_thresholds(entry_price, trd_side)

                # Display execution and threshold details
                print(f"Execution price: {entry_price}")
                print(f"Stop-loss price: {stop_loss_price}")
                print(f"Take-profit price: {take_profit_price}")

                return True

        time.sleep(5)  # Check every 5 seconds

    return False


def monitor_position():
    """Monitor the position for stop loss or take profit."""
    global entry_price, position_side, executed_today

    if entry_price is None or position_side is None:
        return  # No active position to monitor

    # Fetch the current price
    ret, data = quote_context.get_cur_kline(SYMBOL, 1, KLType.K_DAY)
    if ret != RET_OK:
        print(f"Failed to fetch current price: {data}")
        return

    current_price = data['close'].iloc[-1]
    stop_loss_price, take_profit_price = calculate_thresholds(entry_price, position_side)

    print(f"Monitoring position: Current price = {current_price}, Entry price = {entry_price}")
    print(f"Stop-loss price: {stop_loss_price}, Take-profit price: {take_profit_price}")

    # Check stop loss
    if (position_side == TrdSide.BUY and current_price <= stop_loss_price) or \
       (position_side == TrdSide.SELL and current_price >= stop_loss_price):
        print(f"Stop loss triggered at {current_price}")
        close_position()

    # Check take profit
    elif (position_side == TrdSide.BUY and current_price >= take_profit_price) or \
         (position_side == TrdSide.SELL and current_price <= take_profit_price):
        print(f"Take profit triggered at {current_price}")
        close_position()


def close_position():
    """Close the current position."""
    global entry_price, position_side, executed_today

    exit_side = TrdSide.SELL if position_side == TrdSide.BUY else TrdSide.BUY
    ret, data = future_context.place_order(
        price=None,  # Market order
        qty=QUANTITY,
        code=SYMBOL,
        trd_side=exit_side,
        order_type=OrderType.MARKET,
        trd_env=TRADING_ENVIRONMENT
    )
    if ret == RET_OK:
        print(f"Position closed successfully: {data}")
    else:
        print(f"Failed to close position: {data}")

    # Reset position tracking
    entry_price = None
    position_side = None
    executed_today = True  # Ensure no more trades for the day

def calculate_thresholds(execution_price, trd_side):
    """Calculate stop-loss and take-profit prices based on execution price."""
    if trd_side == TrdSide.BUY:
        stop_loss_price = execution_price * (1 - STOP_LOSS_PERCENT)
        take_profit_price = execution_price * (1 + TAKE_PROFIT_PERCENT)
    elif trd_side == TrdSide.SELL:
        stop_loss_price = execution_price * (1 + STOP_LOSS_PERCENT)
        take_profit_price = execution_price * (1 - TAKE_PROFIT_PERCENT)
    else:
        raise ValueError("Invalid trade side for threshold calculation")
    
    return stop_loss_price, take_profit_price


In [11]:
# Main loop
try:
    # Subscribe to daily candlestick data before starting the strategy
    if subscribe_to_klines(SYMBOL):
        while True:
            # Get the current date
            current_date = datetime.now().date()

            # Reset execution tracker if the day has changed
            if current_date != last_trade_date:
                executed_today = False
                entry_price = None
                position_side = None
                last_trade_date = current_date
                print(f"New trading day: {current_date}. Resetting execution tracker.")

            if executed_today:
                print(f"Trade already executed and closed for today ({current_date}). Waiting for the next trading day.")
                # Wait until the next day
                time.sleep((24 * 60 * 60) - datetime.now().time().second)  # Sleep until midnight
                continue

            # Execute trading logic and monitor position
            trading_logic()  # Check for buy/sell signals and place trades
            monitor_position()  # Monitor active positions for stop loss/take profit

            # Pause between iterations to reduce API load
            time.sleep(60)  # Check every 1 minute
except KeyboardInterrupt:
    print("Stopping the trading bot...")
finally:
    # Ensure contexts are closed properly
    quote_context.close()
    future_context.close()

Failed to subscribe to daily candlestick data: Connection closed


In [8]:
# Backtest Periods
ktype = KLType.K_DAY  # K-line type (daily data)
num_klines = 770  # Number of K-lines to fetch
END_DATE = datetime(2024, 6, 30)
START_DATES = {
    "1_year": END_DATE - timedelta(days=365),
    "2_years": END_DATE - timedelta(days=2 * 365),
    "3_years": END_DATE - timedelta(days=3 * 365)
}

def calculate_moving_average(data, lookback):
    """Calculate moving average."""
    return data['close'].rolling(window=lookback).mean()

def calculate_thresholds(entry_price, trd_side):
    """Calculate stop-loss and take-profit levels."""
    if trd_side == "BUY":
        stop_loss_price = entry_price * (1 - STOP_LOSS_PERCENT)
        take_profit_price = entry_price * (1 + TAKE_PROFIT_PERCENT)
    elif trd_side == "SELL":
        stop_loss_price = entry_price * (1 + STOP_LOSS_PERCENT)
        take_profit_price = entry_price * (1 - TAKE_PROFIT_PERCENT)
    return stop_loss_price, take_profit_price

def backtest_strategy(historical_data, start_date, end_date):
    """Backtest the trading strategy."""
    data = historical_data[(historical_data['date'] >= start_date) & (historical_data['date'] <= end_date)].copy()
    data['ma'] = calculate_moving_average(data, LOOKBACK_PERIOD)
    
    trades = []  # Store trade details
    position = None  # Current position
    entry_price = None

    for i in range(len(data)):
        current_row = data.iloc[i]
        current_price = current_row['close']
        ma = current_row['ma']
        date = current_row['date']

        # Skip rows without enough data for MA calculation
        if np.isnan(ma):
            continue

        # Check if we need to close the position
        if position:
            stop_loss_price, take_profit_price = calculate_thresholds(entry_price, position)

            # Check stop-loss or take-profit
            if position == "BUY" and (current_price <= stop_loss_price or current_price >= take_profit_price):
                pnl = current_price - entry_price
                trades.append({
                    "date": date,
                    "position": position,
                    "entry_price": entry_price,
                    "exit_price": current_price,
                    "pnl": pnl
                })
                position = None  # Close position
            elif position == "SELL" and (current_price >= stop_loss_price or current_price <= take_profit_price):
                pnl = entry_price - current_price
                trades.append({
                    "date": date,
                    "position": position,
                    "entry_price": entry_price,
                    "exit_price": current_price,
                    "pnl": pnl
                })
                position = None  # Close position

        # Enter new position if no active trade
        if not position:
            if current_price > ma:
                position = "SELL"
                entry_price = current_price
            elif current_price < ma:
                position = "BUY"
                entry_price = current_price

    return pd.DataFrame(trades)

# Backtest for different periods
results = {}
for period, start_date in START_DATES.items():
    print(f"Fetching data for {period} backtest...")
    historical_data = fetch_historical_data_futu(SYMBOL, start_date, END_DATE)
    trades = backtest_strategy(historical_data, start_date, END_DATE)
    total_pnl = trades['pnl'].sum()
    results[period] = {
        "total_pnl": total_pnl,
        "num_trades": len(trades),
        "trades": trades
    }

# Display results
for period, result in results.items():
    print(f"\nBacktest Results for {period.capitalize()}:")
    print(f"Total P&L: {result['total_pnl']:.2f}")
    print(f"Number of Trades: {result['num_trades']}")
    print(result['trades'].head())

Fetching data for 1_year backtest...


NameError: name 'fetch_historical_data_futu' is not defined

In [15]:
result

{'total_pnl': 1208.0,
 'num_trades': 341,
 'trades':           date position  entry_price  exit_price    pnl
 0   2021-11-26      BUY       6298.0      6127.0 -171.0
 1   2021-11-30      BUY       6127.0      6037.0  -90.0
 2   2021-12-03      BUY       6037.0      5928.0 -109.0
 3   2021-12-06      BUY       5928.0      5695.0 -233.0
 4   2021-12-07      BUY       5695.0      5993.0  298.0
 ..         ...      ...          ...         ...    ...
 336 2024-06-12      BUY       3754.0      3694.0  -60.0
 337 2024-06-19      BUY       3694.0      3839.0  145.0
 338 2024-06-21     SELL       3839.0      3701.0  138.0
 339 2024-06-25      BUY       3701.0      3660.0  -41.0
 340 2024-06-27      BUY       3660.0      3594.0  -66.0
 
 [341 rows x 5 columns]}

In [12]:
import pandas as pd

# Futu Paper Trading Fee Structure
TRANSACTION_FEE_PERCENT = 0.0003  # 0.03% of trade value
FIXED_FEE = 1  # $1 fixed fee per trade
SLIPPAGE_PERCENT = 0.0005  # 0.05% slippage per trade
LOOKBACK_PERIOD = 5  # Mean reversion lookback period
THRESHOLD = 0.01  # 1% threshold for mean reversion signals

def calculate_trade_pnl(entry_price, exit_price, quantity, position):
    """Calculate P&L for a trade, accounting for transaction fees and slippage."""
    # Adjust prices for slippage
    if position == "BUY":
        entry_price += entry_price * SLIPPAGE_PERCENT  # Higher entry price
        exit_price -= exit_price * SLIPPAGE_PERCENT  # Lower exit price
    elif position == "SELL":
        entry_price -= entry_price * SLIPPAGE_PERCENT  # Lower entry price
        exit_price += exit_price * SLIPPAGE_PERCENT  # Higher exit price

    # Calculate raw P&L
    pnl = (exit_price - entry_price) * quantity if position == "BUY" else (entry_price - exit_price) * quantity

    # Calculate transaction costs
    entry_cost = entry_price * quantity * TRANSACTION_FEE_PERCENT + FIXED_FEE
    exit_cost = exit_price * quantity * TRANSACTION_FEE_PERCENT + FIXED_FEE
    total_cost = entry_cost + exit_cost

    # Net P&L after costs
    net_pnl = pnl - total_cost
    return net_pnl, total_cost

def open_vs_past_close_strategy(data, lookback=LOOKBACK_PERIOD, threshold=THRESHOLD):
    """
    Generate buy/sell signals based on comparing today's opening price
    with the average closing price of the past `lookback` days.
    """
    data['mean_close'] = data['close'].rolling(window=lookback).mean()
    data['signal'] = 0  # Initialize signals

    for i in range(lookback, len(data)):
        today_open = data.loc[i, 'open']
        historical_mean_close = data.loc[i, 'mean_close']

        # Generate signals based on threshold
        if today_open > historical_mean_close * (1 + threshold):
            data.loc[i, 'signal'] = -1  # Sell Signal
        elif today_open < historical_mean_close * (1 - threshold):
            data.loc[i, 'signal'] = 1  # Buy Signal

    return data

def backtest_strategy(data, quantity=1):
    """
    Backtest the trading strategy with transaction fees and slippage included.
    """
    trades = []  # Store trade details
    position, entry_price = None, None

    for i, row in data.iterrows():
        today_open = row['open']
        signal = row['signal']
        date = row['date']

        # Check for position exit
        if position:
            # Exit the position if signal is opposite or the next day opens differently
            net_pnl, cost = calculate_trade_pnl(entry_price, today_open, quantity, position)
            trades.append({
                "date": date,
                "position": position,
                "entry_price": entry_price,
                "exit_price": today_open,
                "pnl": net_pnl,
                "cost": cost
            })
            position, entry_price = None, None  # Close position

        # Check for position entry
        if not position:
            if signal == 1:  # Buy Signal
                position, entry_price = "BUY", today_open
            elif signal == -1:  # Sell Signal
                position, entry_price = "SELL", today_open

    return pd.DataFrame(trades)


In [9]:
# Get historical backtesting data
ktype = KLType.K_DAY  # K-line type (daily data)
num_klines = 1000  # Number of K-lines to fetch
end_date = datetime(2024, 6, 30)
start_dates = {
    "1_year": end_date - timedelta(days=365),
    "2_years": end_date - timedelta(days=2 * 365),
    "3_years": end_date - timedelta(days=3 * 365)
}

def fetch_historical_data_futu(symbol, start_date, end_date):
    """Fetch historical data from Futu API."""
    # Subscribe to K-line data
    ret_sub, msg_sub = quote_context.subscribe(symbol, [SubType.K_DAY], subscribe_push=False)
    if ret_sub != RET_OK:
        print(f"Failed to subscribe to {symbol}: {msg_sub}")
        return None  # Exit function if subscription fails

    # Fetch K-line data
    ret_kline, kline_data = quote_context.get_cur_kline(symbol, num_klines, ktype)
    if ret_kline != RET_OK:
        print(f"Failed to fetch K-line data for {symbol}: {kline_data}")
        return None  # Exit function if data fetching fails

    # Process the data
    kline_data.rename(columns={"time_key": "date"}, inplace=True)
    kline_data["date"] = pd.to_datetime(kline_data["date"])  # Ensure date is datetime
    filtered_data = kline_data[(kline_data['date'] >= start_date) & (kline_data['date'] <= end_date)]
    return filtered_data[['date', 'open', 'close']]

In [13]:
results = {}
for period, start_date in start_dates.items():
    print(f"Fetching data for {period} backtest...")
    historical_data = fetch_historical_data_futu(symbol, start_date, end_date)
    if historical_data is None or historical_data.empty:
        print(f"No data available for {period}. Skipping backtest.")
        continue

    # Generate signals
    historical_data = open_vs_past_close_strategy(historical_data)

    # Backtest strategy
    trades = backtest_strategy(historical_data)
    total_pnl = trades['pnl'].sum()
    results[period] = {
        "total_pnl": total_pnl,
        "num_trades": len(trades),
        "trades": trades}

Fetching data for 1_year backtest...


KeyError: 5

In [14]:
historical_data

,date,open,close,mean_close,signal
623,2023-07-03,3909.0,4062.0,NaN,0
624,2023-07-04,4059.0,4088.0,NaN,0
625,2023-07-05,4083.0,4015.0,NaN,0
626,2023-07-06,4009.0,3949.0,NaN,0
627,2023-07-07,3950.0,3894.0,4001.6,0
...,...,...,...,...,...
865,2024-06-24,3699.0,3685.0,3738.2,0
866,2024-06-25,3685.0,3660.0,3730.4,0
867,2024-06-26,3660.0,3686.0,3699.8,0
868,2024-06-27,3696.0,3594.0,3665.2,0


In [42]:
# Apply Strategy
data = open_vs_past_close_strategy(historical_data)
trades = backtest_strategy(historical_data)

# Display Results
print("\nTrading Signals:")
print(data[['date', 'open', 'mean_close', 'signal']])

print("\nBacktest Results:")
print(trades)

KeyError: 5

In [15]:
##### Bollinger Bands #####
def bollinger_bands(data, lookback=30, num_std=2):
    """
    Calculate Bollinger Bands for the given data.
    - data: DataFrame with a 'close' column.
    - lookback: Number of periods to calculate SMA and standard deviation.
    - num_std: Number of standard deviations for the bands.
    Returns:
    - DataFrame with SMA, Upper Band, and Lower Band columns added.
    """
    # Calculate SMA
    data['sma'] = data['close'].rolling(window=lookback).mean()
    # Calculate standard deviation
    data['std'] = data['close'].rolling(window=lookback).std()
    # Calculate Bollinger Bands
    data['upper_band'] = data['sma'] + (num_std * data['std'])
    data['lower_band'] = data['sma'] - (num_std * data['std'])
    return data

def rsi(data, lookback=13):
    """
    Calculate the Relative Strength Index (RSI) for the given data.
    - data: DataFrame with a 'close' column.
    - lookback: Number of periods to calculate RSI.
    Returns:
    - DataFrame with RSI column added.
    """
    # Calculate price changes
    data['change'] = data['close'].diff()
    # Separate gains and losses
    data['gain'] = data['change'].where(data['change'] > 0, 0)
    data['loss'] = -data['change'].where(data['change'] < 0, 0)
    # Calculate average gain and loss
    data['avg_gain'] = data['gain'].rolling(window=lookback).mean()
    data['avg_loss'] = data['loss'].rolling(window=lookback).mean()
    # Compute Relative Strength (RS)
    data['rs'] = data['avg_gain'] / data['avg_loss']
    # Compute RSI
    data['rsi'] = 100 - (100 / (1 + data['rs']))
    # Classify RSI regions
    data['rsi_region'] = data['rsi'].apply(lambda x: 'Overbought' if x > 70 else ('Oversold' if x < 30 else 'Neutral'))
    return data

In [21]:
### Add Bollinger bands and RSI indicators to the dataframe ###
historical_data_technical = historical_data.copy()
historical_data_technical = bollinger_bands(historical_data)
historical_data_technical = rsi(historical_data_technical)
print(historical_data_technical.tail())

          date    open   close  mean_close  signal          sma         std  \
865 2024-06-24  3699.0  3685.0      3738.2       0  3843.000000  136.421507   
866 2024-06-25  3685.0  3660.0      3730.4       0  3832.500000  138.024423   
867 2024-06-26  3660.0  3686.0      3699.8       0  3821.166667  135.524291   
868 2024-06-27  3696.0  3594.0      3665.2       0  3806.100000  134.790092   
869 2024-06-28  3594.0  3569.0      3638.8       0  3789.600000  132.407313   

      upper_band   lower_band  change  gain  loss   avg_gain   avg_loss  \
865  4115.843014  3570.156986   -16.0   0.0  16.0  18.846154  27.076923   
866  4108.548846  3556.451154   -25.0   0.0  25.0  17.769231  29.000000   
867  4092.215248  3550.118086    26.0  26.0  -0.0  17.076923  29.000000   
868  4075.680184  3536.519816   -92.0   0.0  92.0  17.076923  30.384615   
869  4054.414626  3524.785374   -25.0   0.0  25.0  17.076923  31.307692   

           rs        rsi rsi_region  
865  0.696023  41.038526    Neutral 

--- Logging error ---
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 995, in _bootstrap
    self._bootstrap_inner()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/futu/common/network_manager.py", line 338, in _thread_func
    self.poll()
  File "/Library/Framewo

In [ ]:
def divergence(data):
# Initialize variables for tracking divergence
    last_lower_low_price = None
    last_higher_low_rsi = None
    last_higher_high_price = None
    last_lower_high_rsi = None
    data['signal'] = 0

    # Loop through the data to detect divergence
    for i in range(len(data)):
        current_price = data.loc[i, 'close']
        current_rsi = data.loc[i, 'rsi']
        lower_band = data.loc[i, 'upper_band']
        upper_band = data.loc[i, 'lower_band']
        # Skip rows with insufficient data
        if pd.isna(current_rsi) or pd.isna(lower_band) or pd.isna(upper_band):
            continue

        # Detect lower low in price and higher low in RSI (Buy Signal)
        if current_price < lower_band:  # Price is below lower Bollinger Band
            if last_lower_low_price is None or current_price < last_lower_low_price:
                last_lower_low_price = current_price  # Update lower low
            if last_higher_low_rsi is None or current_rsi > last_higher_low_rsi:
                last_higher_low_rsi = current_rsi  # Update higher low in RSI
            if last_lower_low_price is not None and last_higher_low_rsi is not None:
                data.loc[i, 'signal'] = 1  # Buy Signal

        # Detect higher high in price and lower high in RSI (Sell Signal)
        elif current_price > upper_band:  # Price is above upper Bollinger Band
            if last_higher_high_price is None or current_price > last_higher_high_price:
                last_higher_high_price = current_price  # Update higher high
            if last_lower_high_rsi is None or current_rsi < last_lower_high_rsi:
                last_lower_high_rsi = current_rsi  # Update lower high in RSI
            if last_higher_high_price is not None and last_lower_high_rsi is not None:
                data.loc[i, 'signal'] = -1  # Sell Signal

    return data